# SBTi-Finance Tool - Target Reporting
This notebook is an example of how to use the tool to calculate your portfolio's temperature scores and to save results to be submitted to SBTi's Target Validation Team (TVT) for validation and approval of your own Science Based Target. This notebook generates two deliverables to submit to the TVT.

1. Text formatted report in the 'Checks' section of the notebook
2. Two anonymized excel workbooks for reproducing the portfolio temperature scores

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details on how to calculate portfolio temperature scores and how to submit targets to the SBTi for validation and approval. 

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

## Setting up
This section imports SBTi tools, import data providers, and load the portfolio.

In [ ]:
%pip install sbti-finance-tool

In [ ]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd
import requests

### Download resources for Google Colab environment


In [ ]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", "data/example_portfolio.csv")
if not os.path.isfile("utils.py"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/utils.py", "utils.py")

try:  # Import statement when run in Google Colabs
    from utils import  print_aggregations, print_grouped_scores, anonymize, print_percentage_default_scores
except:  # Import statement when run locally
    from examples.utils import print_aggregations, print_grouped_scores, anonymize, print_percentage_default_scores

**Create a data provider**

By running the following cell, you load our example data. If you want to score your own portfolio, this is where you can upload data
from your data provider or your own data lake. Please refer to [Data Requirements](https://sciencebasedtargets.github.io/SBTi-finance-tool/DataRequirements.html#) for a
full description of the data and formats needed.

By specifying a data provider, you can connect to the data source of your choice, e.g. CDP, Urgentem, your own data lake, etc. In this
example, the provider data is located in an Excel file.
Per default we use the example data downloaded in the previous step.
To replace the example data with your own data, first place your file in the data folder:
- When using Google Colab: click the folder logo in the left pane; hover your mouse of the 'data' folder; click the three dots; click upload; select your file.
- When running this notebook locally: place your file in the 'data' folder that is located in the same directory as this notebook.

Next, change the file location between the quotes below: replace 'data_provider_example' with your_filename (don't remove the double quotes in the cell below).

In [ ]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

**Load your portfolio**

In this example, the portfolio data is stored as a CSV file. If you wish to replace the portfolio file with your own portfolio file,
your CSV file should at least have a "company_id" column (the identifier of the company) and a "investment value" column (the amount
of money invested in each of the companies in your portfolio e.g. the value of the shares you hold). Please refer to [Data Requirements](https://sciencebasedtargets.github.io/SBTi-finance-tool/DataRequirements.html#) for a full description of the data and formats needed.
To change the portfolio file, follow the same steps as for the data provider, described above.
Next, change the file location between the quotes below: replace '`example_portfolio`' with your_filename (don't remove the double quotes in the cell below).

In [ ]:
portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")

# Change the column names
portfolio.rename(columns={
    'company_isin': 'isin', 
    'company_lei': 'lei'
}, inplace=True)


# Check for duplicate values in the 'company_id' column
duplicate_ids = portfolio[portfolio.duplicated('company_id', keep=False)]
if not duplicate_ids.empty:
    print("Error: Duplicate values found in the 'company_id' column:")
    print(duplicate_ids)
else:
    print("No duplicate values found in the 'company_id' column.")

The next cell displays the first 5 rows of data in your file.

In [ ]:
portfolio.head(5)

To load the data from the data provider, we need to convert the data of the portfolio.

In [ ]:
companies = SBTi.utils.dataframe_to_portfolio(portfolio)


Import the CTA and set up a SBTi target frame of reference.

In [ ]:
#Provides an absolute frame of reference for SBTi targets so that they are considered as cardinal compared to others in the calculation of temperature scores.
def inject_sbti_validation_for_timeframe_scope_data(amended_portfolio, original_portfolio, debug=True):
    """
    Specially designed for the SBTi tool where amended_portfolio contains multiple rows 
    per company (one for each time frame and scope combination).
    """
    if 'sbti_validated' not in original_portfolio.columns:
        print("⚠ No 'sbti_validated' column found in original portfolio")
        return amended_portfolio
    
    # Store original values before modification
    original_validated_count = original_portfolio['sbti_validated'].sum()
    original_companies_count = len(original_portfolio)
    
    # Get count of unique companies in amended portfolio
    unique_companies_amended = amended_portfolio['company_id'].nunique()
    
    if debug:
        print(f"Original portfolio: {original_companies_count} companies, {original_validated_count} validated")
        print(f"Amended portfolio: {len(amended_portfolio)} rows, {unique_companies_amended} unique companies")
        
        # Check for duplicated rows by company_id
        if len(amended_portfolio) > unique_companies_amended:
            print(f"Multiple rows per company detected in amended portfolio")
            print(f"Rows per company: {len(amended_portfolio) / unique_companies_amended:.2f}")
            
            # Show distribution of time_frame and scope if they exist
            if 'time_frame' in amended_portfolio.columns:
                print("\nTime frame distribution:")
                print(amended_portfolio['time_frame'].value_counts())
            if 'scope' in amended_portfolio.columns:
                print("\nScope distribution:")
                print(amended_portfolio['scope'].value_counts())
    
    # Create a validation mapping
    validation_map = dict(zip(original_portfolio['company_id'], original_portfolio['sbti_validated']))
    
    # Apply validation to all rows in amended portfolio
    amended_portfolio['sbti_validated'] = amended_portfolio['company_id'].map(validation_map).fillna(False)
    
    # Count unique validated companies after modification
    validated_companies = amended_portfolio[amended_portfolio['sbti_validated']]['company_id'].nunique()
    
    # Print validation summary
    print(f"\nOriginal validated companies: {original_validated_count}")
    print(f"Unique companies validated in amended portfolio: {validated_companies}")
    
    if original_validated_count == validated_companies:
        print("✓ CTA validation successfully preserved at company level")
    else:
        print("⚠ CTA validation mismatch at company level")
        
        # Additional debugging
        if debug:
            print("\nChecking for specific discrepancies...")
            # Get list of company IDs that should be validated
            original_validated_ids = set(original_portfolio[original_portfolio['sbti_validated']]['company_id'])
            amended_validated_ids = set(amended_portfolio[amended_portfolio['sbti_validated']]['company_id'])
            
            missing_validations = original_validated_ids - amended_validated_ids
            extra_validations = amended_validated_ids - original_validated_ids
            
            if missing_validations:
                print(f"Companies that should be validated but aren't: {len(missing_validations)}")
                print(missing_validations)
            
            if extra_validations:
                print(f"Companies that shouldn't be validated but are: {len(extra_validations)}")
                print(extra_validations)
    
    return amended_portfolio
# STANDALONE SBTi VALIDATION - Download and process CTA data
print("Downloading SBTi Companies Taking Action (CTA) data...")
CTA_FILE_URL = "https://cdn.sciencebasedtargets.org/download/target-dashboard"

try:
    resp = requests.get(CTA_FILE_URL)
    if resp.status_code == 200:
        cta_file = pd.read_excel(resp.content)
        print(f"Downloaded CTA data with {len(cta_file)} rows")
        
        # Extract relevant columns
        targets = cta_file[['company_name', 'isin', 'lei', 'action', 'target', 'date_published']]
        
        # Filter for companies with targets
        companies_with_targets = targets[targets['action'] == 'Target']
        
        # Get unique identifiers
        all_isin_set = set(companies_with_targets['isin'].dropna())
        all_lei_set = set(companies_with_targets['lei'].dropna())
        
        # Create a set of lowercase company names
        companies_with_targets['company_name_lower'] = companies_with_targets['company_name'].str.lower()
        company_name_set = set(companies_with_targets['company_name_lower'].dropna())
        
        # Add portfolio columns if they don't exist
        if 'isin' not in portfolio.columns and 'company_isin' in portfolio.columns:
            portfolio['isin'] = portfolio['company_isin']
        if 'lei' not in portfolio.columns and 'company_lei' in portfolio.columns:
            portfolio['lei'] = portfolio['company_lei']
        
        # Function to check if ISIN, LEI, or company name is validated
        def is_validated(row):
            # First check LEI
            if pd.notna(row.get('lei')) and str(row.get('lei')).lower() in [str(x).lower() for x in all_lei_set]:
                return True
            
            # Then check ISIN
            if pd.notna(row.get('isin')) and str(row.get('isin')).lower() in [str(x).lower() for x in all_isin_set]:
                return True
            
            # Finally check company name
            if pd.notna(row.get('company_name')):
                company_name_lower = str(row.get('company_name')).lower()
                if company_name_lower in company_name_set:
                    return True
            
            return False
        
        # Add the validated column to the portfolio
        portfolio['sbti_validated'] = portfolio.apply(is_validated, axis=1)
        
        # Convert portfolio to company objects again (after adding sbti_validated)
        companies = SBTi.utils.dataframe_to_portfolio(portfolio)
        
        # Print validation summary
        validated_count = portfolio['sbti_validated'].sum()
        print(f"Companies with SBTi-validated targets: {validated_count} out of {len(portfolio)} ({validated_count/len(portfolio)*100:.2f}%)")
        
        if 'investment_value' in portfolio.columns:
            total_investment = portfolio['investment_value'].sum()
            validated_investment = portfolio[portfolio['sbti_validated']]['investment_value'].sum()
            print(f"Portfolio coverage by investment value: {validated_investment/total_investment*100:.2f}%")
    else:
        print(f"Failed to download CTA file: HTTP {resp.status_code}")
except Exception as e:
    print(f"Error processing CTA file: {str(e)}")

# Update provider data with our validation results
print("Updating provider data with validated companies...")
try:
    # Get the fundamental_data from provider
    if hasattr(provider, 'data') and 'fundamental_data' in provider.data:
        # Create a mapping of company_id to validation status
        validation_map = portfolio[['company_id', 'sbti_validated']].set_index('company_id')['sbti_validated'].to_dict()
        
        # Update sbti_validated in provider data
        updated_count = 0
        for idx, row in provider.data['fundamental_data'].iterrows():
            company_id = row['company_id']
            if company_id in validation_map:
                # Ensure we're setting a proper boolean value
                is_validated = bool(validation_map[company_id])
                provider.data['fundamental_data'].at[idx, 'sbti_validated'] = is_validated
                updated_count += 1
        
        # Force the sbti_validated column to be boolean type
        provider.data['fundamental_data']['sbti_validated'] = provider.data['fundamental_data']['sbti_validated'].astype(bool)
        
        print(f"Updated sbti_validated for {updated_count} companies in provider data")
    else:
        print("Provider does not have expected data structure - sbti_validated flags may be overwritten")
except Exception as e:
    print(f"Error updating provider data: {str(e)}")

## Select calculation options
Configure computational settings for the temperature score. 
You can change the default score (fallback score in the code) and aggregation method below. The values specified here are the recommended defaults by the SBTi. If you want to overwrite the recommended settings see the comments for all the options per setting

In [ ]:
# Settings                                              # Other Options: 
time_frames = [SBTi.interfaces.ETimeFrames.MID]         # reporting to TVT requires 'MID'
scopes = [EScope.S1S2, EScope.S1S2S3]                   # reporting to TVT requires 'S1S2' and 'S1S2S3'
fallback_score = 3.2                                    # The options of the default score are 3.2, 3.9 or 4.5
aggregation_method = PortfolioAggregationMethod.WATS    # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
grouping = ['sector']                                   # reporting to TVT requires 'sector'

In [ ]:
temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    aggregation_method=aggregation_method,
    grouping=grouping
)

## Calculate company and portfolio temperature scores

In [ ]:
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)
# Preserve the CTA validation from our direct download
amended_portfolio = inject_sbti_validation_for_timeframe_scope_data(amended_portfolio, portfolio, debug=True)
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
amended_portfolio.head(5) # display top 5 rows

## Checks
In the cell below all figures are printed in a text format one can copy and send for validation purposes to the TVT. 
- Calculation settings
- Portfolio coverage
- Portfolio temperature score
- Percentage covered by targets vs. default scores
- Temperature Score for each sector



In [ ]:

# Portfolio temperature score
print('Calculation settings:')
print()
print('Aggregation method:\t {}'.format(str(aggregation_method).split('.')[1]))
print('Default score: \t\t {:.2f}'.format(fallback_score))
print()

# Portfolio coverage: Percentage of SBTs in portfolio
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Portfolio coverage is {c:.2f}%".format(c=coverage))
print()

print('Portfolio Temperature scores:\n')
print_aggregations(aggregated_scores)
print()

# Percentage of score based on default score
print('Percentage of score based on default:\n')
print_percentage_default_scores(aggregated_scores)
print()

# Temperature score per sector
print('Temperature scores per sector:')
print_grouped_scores(aggregated_scores)




## Save anonymized data for SBTi target validation
In order for the targets to be validated by SBTi, you can save your data locally. By running the anonymize function, you can replace company identifiers with meaningless substitutes.

In [ ]:
portfolio, provider = anonymize(portfolio, provider)

In order to store the portfolio and provider data locally, two options apply:
1. You are running the SBTi tool locally or from Google Colab
2. You are running the SBTi tool from a Docker container

If you run the SBTi tool locally or from Google Colab, you:
- Specify and filenames in the cell below
- Run the cell below

In [ ]:
portfolio_filename = 'portfolio.xlsx'
provider_filename = 'provider.xlsx'
portfolio.to_excel(portfolio_filename, index=False)

with pd.ExcelWriter(provider_filename, engine='openpyxl') as writer:
    provider.data['fundamental_data'].to_excel(writer, sheet_name='fundamental_data')
    provider.data['target_data'].to_excel(writer, sheet_name='target_data')


If you run the SBTi tool locally, you find the output files in the root folder of this notebook

If you run the SBTi tool from Google Colab, you:
- Click on the files icon in the left pane
- Click the three dots that appear after hovering over the file
- Download the file to your local machine